# Sample pipeline for text feature extraction and evaluation

Adapted from http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

The dataset used in this example is the 20 newsgroups dataset which will be automatically downloaded and then cached and reused for the document classification example.

You can adjust the number of categories by giving their names to the dataset loader or setting them to None to get the 20 of them.

In [1]:
include("preamble.jl")

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at /Applications/Julia-0.4.3.app/Contents/Resources/julia/lib/julia/sys.dylib
 in vect at abstractarray.jl:38
 in find_in_path at /Users/cedric/.julia/v0.4/Autoreload/src/files.jl:11
 in find_file at /Users/cedric/.julia/v0.4/Autoreload/src/files.jl:40
 in arequire at /Users/cedric/.julia/v0.4/Autoreload/src/Autoreload.jl:50
 in include at /Applications/Julia-0.4.3.app/Contents/Resources/julia/lib/julia/sys.dylib
 in include_from_node1 at /Applications/Julia-0.4.3.app/Contents/Resources/julia/lib/julia/sys.dylib
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/cedric/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/cedric/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /Users/cedric/Programa/Sklearn/notebooks/preamble.jl, in expression starting on line 7
 in depwarn at deprecated.jl:73
 [inlined code] from deprecated.jl:694
 in require at no file:0

In [2]:
# Python Version Authors: Olivier Grisel <olivier.grisel@ensta.org>
#                         Peter Prettenhofer <peter.prettenhofer@gmail.com>
#                         Mathieu Blondel <mathieu@mblondel.org>
# License: BSD 3 clause

@pyimport2 sklearn.datasets: fetch_20newsgroups
@pyimport2 sklearn.feature_extraction.text: CountVectorizer
@pyimport2 sklearn.feature_extraction.text: TfidfTransformer
@pyimport2 sklearn.linear_model: SGDClassifier
@pyimport2 sklearn.grid_search: GridSearchCV
@pyimport2 sklearn.pipeline: Pipeline

###############################################################################
# Load some categories from the training set
categories = [
    "alt.atheism",
    "talk.religion.misc",
]
# Uncomment the following to do the analysis on all the categories
#categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

data = fetch_20newsgroups(subset="train", categories=categories)
filenames = data["filenames"]
target_names = data["target_names"]
println("$(length(filenames)) documents")
println("$(length(target_names)) categories")
println()

cv = CountVectorizer()
fit_transform!(cv, data["data"][1:5], [0,0,1])

###############################################################################
# define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Skcore.Pipeline([
    ("vect", CountVectorizer()),
    ("tfidf", TfidfTransformer()),
    ("clf", SGDClassifier()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = Dict(
    "vect__max_df"=> (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range"=> ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    "clf__alpha"=> (0.00001, 0.000001),
    "clf__penalty"=> ("l2", "elasticnet"),
    #'clf__n_iter': (10, 50, 80),
)

# find the best parameters for both the feature extraction and the
# classifier
grid_search = Skcore.GridSearchCV(pipeline, parameters, n_jobs=1, verbose=1)

println("Performing grid search...")
println("pipeline:", [name for (name, _) in pipeline.steps])
println("parameters:")
println(parameters)
t0 = time()
N_samples = 100   # number of posts to train on
fit!(grid_search, data["data"][1:N_samples], data["target"][1:N_samples])
@printf("done in %0.3fs", time() - t0)
println()

@printf("Best score: %0.3f\n", grid_search.best_score_)
@printf("Best parameters set:\n")
best_parameters = get_params(grid_search.best_estimator_)
for param_name in keys(parameters)
    println("\t$param_name: $(best_parameters[param_name])")
end

Loading 20 newsgroups dataset for categories:ASCIIString

No handlers could be found for logger "sklearn.datasets.twenty_newsgroups"


["alt.atheism","talk.religion.misc"]857 documents
2 categories

